This notebook evaluates the top taxonomy labels for each document by querying ChatGPT to determine if each label is a suitable fit. The results are stored as a new column in the DataFrame.

In [0]:
%run ./utils

In [0]:
%run ./config

In [0]:
import pandas as pd
import numpy as np
import openai
import json
from concurrent.futures import ThreadPoolExecutor, as_completed  
import re  
import pickle
import math
from collections import Counter
import random  
import json  
import time

In [0]:
conf = DataBricksDevConfig

path_results = conf.Data.path_results
path_taxonomy = conf.Data.path_taxonomy
folder_name = conf.Data.folder_name
dic_abbreviation2full = conf.taxonomy_info.dic_abbreviation2full

In [0]:
try:
    df_in = pd.read_pickle(path_results + '6-SelectionReduction.pckl')
    print(len(df_in))
    df_in.head(2)
except Exception as e:
    print("Cluster could not read the file (1- wrong cluster 2- no new json file)",e)
    dbutils.notebook.exit(str(e))  

## Read Taxonomy from generated Dataframe

In [0]:

with open(path_taxonomy + 'dic_taxonomy_embeddings.pickle', 'rb') as file:
    dic_taxonomy = pickle.load(file)
#1
dic_taxonomy_id2label = {dic_taxonomy[label]['id']:label for label in dic_taxonomy.keys()}
#2   
labels_all = list(dic_taxonomy.keys())  


In [0]:

def clean_label(label):
    # return((label+''))    
    cleaned_label = label
    for abbreviation, expansion in dic_abbreviation2full.items():
        # if abbreviation in cleaned_label:
        if cleaned_label.startswith(abbreviation):
            cleaned_label = cleaned_label.replace(abbreviation, expansion)
    return cleaned_label  


for k,v in dic_taxonomy.items():
    dic_taxonomy[k]['cleaned_name']= clean_label(k)



# required functions

In [0]:
import random
def F_select_random_items( labels):
    if len(labels) <= 5:
        return labels
    else:
        return random.sample(labels, 5)

## ASK GPT In parallel

In [0]:
columns_to_check = ['labels_Proposed_approach']
for col in columns_to_check :
    df_in[col+'_ablation3_5labels'] = [[] for _ in range(len(df_in))]

In [0]:

for i_doc,doc_tmp in df_in.iloc[:].iterrows():
    print(i_doc, end = ', ')
    for col in columns_to_check:
        print(col, end = ' ')
        # call GPT in parallel
        selected_labels = doc_tmp[col]
        # print(i_doc)
        if len(selected_labels) > 5 :
            selected_labels = F_select_random_items(selected_labels)
            
        df_in.at[i_doc,col+'_ablation3_5labels'] = selected_labels
    print('')

# Store


In [0]:
df_in.to_pickle(path_results + f'6-SelectionReduction_ablation3.pckl')